<a href="https://colab.research.google.com/github/CVasquezroque/Codigos/blob/master/Pr%C3%A1ctica_Caracterizando_una_Se%C3%B1al_Biom%C3%A9dica.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Universidad Peruana Cayetano Heredia**
# **Introducción a Señales biomédicas (2020-1)**
# **Caracterizando una señal biomédica**

## **Análisis de una onda ECG**

Para el desarrollo de esta práctica necesitaremos:


*   Onda de electrocardiograma proveniente de la libreria SCIPY (scipy.misc [Miscellaneous routines](https://docs.scipy.org/doc/scipy/reference/generated/scipy.misc.electrocardiogram.html))
*   La transformada de Fourier ([Fast Fourier Transform](https://docs.scipy.org/doc/scipy/reference/tutorial/fft.html))
*   Librería de visualización de señales Matplotlib ([Pyplot](https://matplotlib.org/api/pyplot_summary.html))
*   Librería de soporte y operación de matrices y vectores ([Numpy](https://numpy.org/))

**Nota**: Este notebook tiene partes por completar



In [0]:
from statsmodels.graphics.tsaplots import plot_acf
from scipy.fft import fft
from scipy.misc import electrocardiogram
from scipy.stats import kstest
import matplotlib.pyplot as plt
import numpy as np

### **1.1 La señal a analizar: Onda ECG**

Las ondas ECG son señales complejas compuestas por multiples ondas provenientes de la acción potencial de diferentes partes del corazón que son captadas a través de equipos de electrocardiogramas. Los electrocardiogramas captan estas señales mediante el uso de electrodos que pueden ser superficiales (ver figura 1) o intramusculares mediante agujas (ver figura 2). 

![Figura 1. Electrodo Superficial, Referencia: [Adafruit](https://www.adafruit.com/product/2773)](https://cdn-shop.adafruit.com/970x728/2773-00.jpg)

**Figura 1.** Electrodo Superficial, Referencia: [Adafruit](https://www.adafruit.com/product/2773)

![Figura 2. Electrodo Aguja](https://www.adinstruments.com/sites/default/files/products/prod_MLA1203.jpg)

**Figura 2.** Electrodo tipo aguja, Referencia : [ADInstrument](https://www.adinstruments.com/products/needle-electrodes)



#### **ECG data**

La señal de ECG que utilizaremos proviene de la librería MISC de SCIPY, la cual a su vez toma la señal de la base de datos PhysioNet, the MIT-BIH Arrhythmia Database, y han sido digitalizadas con una frecuencia de muestreo de 360 Hz por un tiempo de 5 minutos.

De manera general, podemos extraer algunos parámetros de la señal como:


*   Tamaño del array:


```
ecg.shape
```


*   El valor medio de la onda:


```
ecg.mean()
```


*   La desviación estandar de la onda:


```
ecg.std()
```




**Preguntas**

*   Una señal de 5 minutos, muestreada a 360Hz ¿cuántas muestras debería tener?





In [0]:
?electrocardiogram

Vamos a cargar la señal en la variable `ecg` y vamos a ver el tipo de dato tiene. Un `numpy.ndarray` es el objeto usado para representar vectores, matrices y otros datos n-dimensionales.

In [0]:
ecg = electrocardiogram()
type(ecg)

A continuación, vamos a usar la función `matplotlib.pyplot.plot` para mostrar gráficamente la señal.

In [0]:
fs = 360
time = np.arange(ecg.size) / fs
plt.plot(time, ecg)
plt.xlabel("tiempo en s")
plt.ylabel("ECG en mV")
plt.xlim(50, 52)
plt.ylim(-1, 1.5)
plt.show()

Para poder visualizar mejor la señal podemos obtener un "slice" del vector usando `ecg[:500]` y mostrarlo usando la función `matplotlib.pyplot.plot`.

In [0]:
plt.figure()
plt.title("ECG")  #titulo del plot
plt.xlabel("Muestras") #etiqueta para el eje x
plt.ylabel("Voltage")  #etiqueta para el eje y
plt.plot(ecg[:500]) #solo mostramos las primeras 500 muestras
plt.show()

### Pregunta

¿Cuál es el tiempo entre los dos picos?. Recuerde que puede usar el `numpy.max` y `numpy.argmax` para ubicar los picos. Recuerde que puede usar `?` para ver la documentación de las funciones

In [0]:
?np.max

In [0]:
?np.argmax

In [0]:
#########################################
######## Escriba su código aquí #########
#########################################

pico1 = np.argmax(ecg[:200])
pico2 = 200 + np.argmax(ecg[200:500])

#########################################
n_muestras = pico2 - pico1 + 1
tiempo = n_muestras/360 #cantidad de muestras / cantidad_de_muestras_por_segundo
print("{:.2f} segundos".format(tiempo))


### Autocorrelación

La correlación nos permite medir el grado de similaridad entre dos señales, para eso usaremos la función `numpy.correlate`

In [0]:
?np.correlate

In [0]:
plt.figure(figsize=(10,10))
plt.suptitle("Correlación")
plt.subplot(2, 2, 1)
plt.title("ECG")
plt.plot(ecg[:1000])
plt.subplot(2, 2, 2)
plt.title("Segmento")
plt.plot(ecg[660:870])
plt.subplot(2, 2, 3)
plt.title("Correlación")
plt.plot(np.correlate(ecg[:1000], ecg[660:870], mode='same'))

### Verificando Normalidad

Una de las características que quisieramos conocer de una señal es si sus datos provienen de una distribución normal.

In [0]:
plt.figure()
plt.title("Histograma de valores de la señal ECG")
plt.hist(ecg, bins=100)
plt.plot()

¿Cuál es el signficado de la prueba Kolmogorov-Smirnov a continuación? 


####################

Escriba aquí

####################

In [0]:
kstest(ecg, 'norm', args=(0,1)) 

La hipótesis nula, es que la distribución de los datos es idéntica a una distribución normal (media=0, std=1). Si el valor es menor a 0.05 podemos rechazar la hipótesis nula y concluir que nuestra muestra no es identica a una distribución normal con los parámetros mencionados.

### **1.2 Fast Fourier transforms**

#### **Transformada Discreta de Fourier**

Fast Fourier Transform (FFT) es un algoritmo matemático que calcula la Transformación Discreta de Fourier (DFT) de una secuencia dada. La única diferencia entre la Transformada de Fourier (FT) y FFT es que FT considera una señal continua mientras que FFT toma una señal discreta como entrada. DFT convierte una secuencia (señal discreta) en sus componentes de frecuencia al igual que FT para una señal continua. En nuestro caso, tenemos una secuencia de amplitudes que se tomaron muestras de una señal de audio continua. El algoritmo DFT o FFT puede convertir esta señal discreta de dominio de tiempo en un dominio de frecuencia.

#### **Prueba con una señal de entrada: Suma de 2 funciones senoidales**

Con esta señal inicial conocida, la cual sabemos su transformada de Fourier, probamos que nuestro código se comporta de manera correcta.

In [0]:
# Número de muestras 
N = 600
# Periodo de muestreo en base a una frecuencia de muestreo de 800Hz
T = 1.0 / 800.0
x = np.linspace(0.0, N*T, N)
# Suma de funciones senoidales de frecuencias de 50 y 80 Hz respectivamente
y = np.sin(50.0 * 2.0*np.pi*x) + 0.5*np.sin(80.0 * 2.0*np.pi*x)
plt.plot(x,y)
plt.grid()
plt.show()

In [0]:
yf = fft(y)
xf = np.linspace(0.0, 1.0/(2.0*T), N//2)
plt.plot(xf, 2.0/N * np.abs(yf[0:N//2]))
plt.grid()
plt.show()

In [0]:
T = 1/fs
N = len(ecg)
yf = fft(ecg)
xf = np.linspace(0.0, 1.0/(2.0*T), N//2)
plt.plot(xf, 2.0/N * np.abs(yf[0:N//2]))
plt.grid()
plt.xlabel("Frecuencia")
plt.ylabel("Magnitud")
plt.show()

## **2. Análisis Estacionario de una onda ECG**

Para el desarrollo de esta práctica necesitaremos:


*   Onda de electrocardiograma proveniente de la libreria SCIPY (scipy.misc [Miscellaneous routines](https://docs.scipy.org/doc/scipy/reference/generated/scipy.misc.electrocardiogram.html))
*   La transformada de Fourier ([Fast Fourier Transform](https://docs.scipy.org/doc/scipy/reference/tutorial/fft.html))
*   Librería de visualización de señales Matplotlib ([Pyplot](https://matplotlib.org/api/pyplot_summary.html))
*   Librería de soporte y operación de matrices y vectores ([Numpy](https://numpy.org/))


Ahora veremos algunos métodos para poder establecer la estacionariedad de la señal.

### **2.1. Visual**

La primera forma de analizar la estacionariedad de la señal es el análisis visual de la señal. 

In [0]:
fs = 360
time = np.arange(ecg.size) / fs
plt.plot(time, ecg)
plt.xlabel("tiempo en s")
plt.ylabel("ECG en mV")
#plt.xlim(50, 52)
#plt.ylim(-1, 1.5)
plt.show()

In [0]:
fs = 360
time = np.arange(ecg.size) / fs
plt.plot(time, ecg)
plt.xlabel("tiempo en s")
plt.ylabel("ECG en mV")
#########################################
######## Escriba su código aquí #########
#########################################
# Objetivo: mostrar una parte de la señal
plt.show()

### **2.2. Prueba estadística**

En lugar de realizar la prueba visual, podemos usar pruebas estadísticas como las pruebas estacionarias de raíz unitaria. La raíz unitaria indica que las propiedades estadísticas de una serie dada no son constantes con el tiempo, que es la condición para las series temporales estacionarias. Aquí está la explicación matemática de lo mismo: 

Supongamos que tenemos una serie: 
$$y_{t} = a*y_{t-1} + \epsilon_{t}$$, donde $y_t$ es el valor a un instante de tiempo y $\epsilon_{t}$ es el error. Para calcular $y_{t}$ necesitamos tener primero $y_{t-1}$.

$$y_{t-1} = a*y_{t-2} + \epsilon_{t-1}$$

Si hacemos eso para todas las observaciones, el valor de $y_{t}$ resultará ser:

$$y_{t} = a^n*y_{t-n} + \sum\epsilon_{t-i}*a^i$$

Si el valor de a es 1 (unidad) en la ecuación anterior, las predicciones serán iguales a $y_{t-n}$ y la suma de todos los errores de t-n a t, lo que significa que la varianza aumentará con el tiempo. Esto se conoce como raíz unitaria en una serie de tiempo. Sabemos que para una serie temporal estacionaria, la varianza no debe ser una función del tiempo. Las pruebas de raíz unitaria verifican la presencia de raíz unitaria en la serie verificando si el valor de a = 1. A continuación se presentan las dos pruebas estacionarias de raíz unitaria más utilizadas:

#### **Augmented Dickey Fuller (ADF)**

La prueba Dickey Fuller es una de las pruebas estadísticas más populares. Se puede utilizar para determinar la presencia de raíz unitaria en la serie y, por lo tanto, ayudarnos a comprender si la serie es estacionaria o no. Las hipótesis nula y alternativa de esta prueba son:

**Hipótesis nula:** la serie tiene una raíz unitaria (valor de a = 1)

**Hipótesis alternativa:** la serie no tiene raíz unitaria.

Si no podemos rechazar la hipótesis nula, podemos decir que la serie no es estacionaria. Esto significa que la serie puede ser lineal o estacionaria en diferencias (entenderemos más sobre la diferencia estacionaria en la siguiente sección).

In [0]:
#Definimos una funcion para el ADF test
import pandas as pd
from statsmodels.tsa.stattools import adfuller
def adf_test(timeseries):
    #Perform Dickey-Fuller test:
    print ('Resultados del Dickey-Fuller Test:')
    dftest = adfuller(timeseries, autolag='AIC')
    dfoutput = pd.Series(dftest[0:4], index=['Test Statistic','p-value','#Lags Used','Número de Observaciones Usados'])
    for key,value in dftest[4].items():
       dfoutput['Valores Críticos (%s)'%key] = value
    print (dfoutput)

#Aplicamos adf test
adf_test(ecg)

**Resultados de la prueba de ADF:** Las pruebas de ADF dan los siguientes resultados: Test Statistic, p-value y el valor crítico a intervalos de confianza del 1%, 5% y 10%. Los resultados de nuestra prueba para esta serie en particular son:

**Prueba de estacionariedad:** si el p-value es menor a 0.05, entonces podemos rechazar la hipotesis nula.

En nuestro ejemplo anterior, p-value < 0.05, lo que implica que podemos rechazar la hipotesis nula. Entonces la señal es ...

#### **Kwiatkowski-Phillips-Schmidt-Shin (KPSS)**

KPSS es otra prueba para verificar la estacionariedad de una serie de tiempo (un poco menos popular que la prueba de Dickey Fuller). La hipótesis nula y alternativa para la prueba KPSS es opuesta a la de la prueba ADF, que a menudo crea confusión.

Los autores de la prueba KPSS han definido la hipótesis nula ya que el proceso es estacionario de tendencia, a una hipótesis alternativa de una serie de raíz unitaria. Entenderemos la tendencia estacionaria en detalle en la siguiente sección. Por ahora, centrémonos en la implementación y veamos los resultados de la prueba KPSS.

**Hipótesis nula:** el proceso es de tendencia estacionaria.

**Hipótesis alternativa:** la serie tiene una raíz unitaria (la serie no es estacionaria).

In [0]:
#Definimos la función para kpss test
from statsmodels.tsa.stattools import kpss
def kpss_test(timeseries):
    print ('Resultados de KPSS Test:')
    kpsstest = kpss(timeseries, regression='c')
    kpss_output = pd.Series(kpsstest[0:3], index=['Test Statistic','p-value','Lags Used'])
    for key,value in kpsstest[3].items():
      kpss_output['Valor Crítico (%s)'%key] = value
    print (kpss_output)

kpss_test(ecg)

**Resultados de la prueba KPSS:** Los siguientes son los resultados de la prueba KPSS: Test Statistic, valor p y valor crítico a intervalos de confianza de 1%, 2.5%, 5% y 10%.

**Prueba de estacionariedad:** si el Test Statistic es mayor que el valor crítico, rechazamos la hipótesis nula (la serie no es estacionaria). Si el Test Statistic es menor que el valor crítico, si no se rechaza la hipótesis nula (la serie es estacionaria).

Para la señal de ECG, el valor del Test Statistic < el valor crítico en todos los intervalos de confianza, y por lo tanto podemos decir que la serie es estacionaria.

Si ambas pruebas mostraron resultados contradictorios. Una de las pruebas podría mostrar que la serie es estacionaria, mientras que la otra que la serie no lo es. Tenemos que tener en cuenta que hay más de un tipo de estacionariedad.

En resumen, la prueba ADF tiene una hipótesis alternativa de lineal o diferencia estacionaria, mientras que la prueba KPSS identifica la tendencia-estacionariedad en una serie.

#### **Tipos de estacionariedad**
Comprendamos los diferentes tipos de estacionariedad y cómo interpretar los resultados de las pruebas anteriores.

**Estacionario estricto:** una serie estacionaria estricta satisface la definición matemática de un proceso estacionario. Para una serie estacionaria estricta, la media, la varianza y la covarianza no son función del tiempo. El objetivo es convertir una serie no estacionaria en una serie estacionaria estricta para hacer predicciones.

**Tendencia estacionaria:** una serie que no tiene raíz unitaria pero exhibe una tendencia se denomina serie estacionaria de tendencia. Una vez que se elimina la tendencia, la serie resultante será estrictamente estacionaria. La prueba KPSS clasifica una serie como estacionaria en ausencia de raíz unitaria. Esto significa que la serie puede ser estacionaria estricta o estacionaria de tendencia.

**Diferencia estacionaria:** una serie de tiempo que puede hacerse estrictamente estacionaria por diferenciación cae bajo diferencia estacionaria. La prueba ADF también se conoce como prueba de estacionariedad de diferencia.


Siempre es mejor aplicar ambas pruebas, de modo que estemos seguros de que la serie es realmente estacionaria. Veamos los posibles resultados de la aplicación de estas pruebas estacionarias.

**Caso 1:** Ambas pruebas concluyen que la serie no es estacionaria -> la serie no es estacionaria

**Caso 2:** Ambas pruebas concluyen que la serie es estacionaria -> la serie es estacionaria

**Caso 3:** KPSS = estacionario y ADF = no estacionario -> tendencia estacionaria, elimine la tendencia para hacer series estrictamente estacionarias

**Caso 4:** KPSS = no estacionario y ADF = estacionario -> diferencia estacionaria, use la diferenciación para hacer estacionarias en serie